In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
def plot_multiple(images, rc):
    if rc == 'row':
        fig, axes = plt.subplots(nrows=len(images), ncols=1, figsize=(14, 14))
        for ax, img in zip(axes, images):
            ax.imshow(img, cmap='gray')
            ax.axis('off')
        plt.show()
    elif rc == 'column':
        fig, axes = plt.subplots(nrows=1, ncols=len(images), figsize=(14, 14))
        for ax, img in zip(axes, images):
            ax.imshow(img, cmap='gray')
            ax.axis('off')
        plt.show()
    else:
        print("rc is not valid!")


In [ ]:
def slice_vertical(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Uses Otsu method for thresholding the image 
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Detect vertical lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 80))
    detect_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(detect_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]   # Why?

    cnts_x_averages = []
    for cnt in cnts:
        temp_total = 0
        for pnt in cnt:
            temp_total += pnt[0][0]
        cnts_x_averages.append(temp_total / len(cnt))
    cnts_x_averages.sort()
    cnts_x_averages_differences = [abs(cnts_x_averages[i] - cnts_x_averages[i + 1]) for i in range(len(cnts_x_averages) - 1)]
    cnts_x_averages_differences.sort()
    average_difference_offset = int(sum(cnts_x_averages_differences[-4:]) / len(cnts_x_averages_differences[-4:])) - 50 # offset

    sliced_images = []
    for i in range(1, len(cnts_x_averages)):
        if cnts_x_averages[i] - cnts_x_averages[i - 1] > average_difference_offset:
            temp_sliced_image = image[ : len(image[0]), int(cnts_x_averages[i - 1]) : int(cnts_x_averages[i])]
            sliced_images.append(temp_sliced_image)

    return sliced_images

In [ ]:
def slice_horizontal(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Uses Otsu method for thresholding the image 
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Detect horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (80,1))
    detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]   # Why?

    cnts_y_averages = []
    for cnt in cnts:
        temp_total = 0
        for pnt in cnt:
            temp_total += pnt[0][1]
        cnts_y_averages.append(temp_total / len(cnt))
    cnts_y_averages.sort()

    filtered_cnts_y_averages = [cnts_y_averages[0]]
    for i in range(1, len(cnts_y_averages)):
        if cnts_y_averages[i] > cnts_y_averages[i - 1] + 20: # offset
            filtered_cnts_y_averages.append(cnts_y_averages[i])

    sliced_images = []
    for i in range(1, len(filtered_cnts_y_averages)):
        temp_sliced_image = image[int(filtered_cnts_y_averages[i - 1]) : int(filtered_cnts_y_averages[i]), : len(image[0])]
        sliced_images.append(temp_sliced_image)

    return sliced_images


In [ ]:
def predict_move(image):
    # ##
    width_cut_offset = 15
    height_cut_offset = 10
    cut_image = image[height_cut_offset : len(image) - height_cut_offset, width_cut_offset : len(image[0]) - width_cut_offset]

    # ##
    gray = cv2.cvtColor(cut_image, cv2.COLOR_BGR2GRAY)

    # Uses Otsu method for thresholding the image 
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)

    # Add white border to improve contour find algorithm
    thresh = cv2.copyMakeBorder(thresh, height_cut_offset, height_cut_offset, width_cut_offset, width_cut_offset, cv2.BORDER_CONSTANT, value=(255, 255, 255))

    # Create an empty white image with the same size as the original image
    # thresh[ : width_border_size, : len(thresh[0])] = 255
    # thresh[ : len(thresh), len(thresh[0]) - height_border_size : len(thresh[0])] = 255
    # thresh[len(thresh) - width_border_size : len(thresh), : len(thresh[0])] = 255
    # thresh[ : len(thresh), : height_border_size] = 255

    ctrs, _ = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

    # Calculate average bounding contour area
    areas = [cv2.contourArea(ctr) for ctr in ctrs]
    average_area = sum(areas) / len(areas)

    # Define thresholds as a percentage of the average area
    lower_threshold = 500  # 0.5 * average_area
    upper_threshold = 6400 # 6.0 * average_area

    extracted_symbols = []
    for i, ctr in enumerate(sorted_ctrs):
        x, y, w, h = cv2.boundingRect(ctr)
        area = w*h
        if lower_threshold < area < upper_threshold:
            extracted_symbols.append(thresh[y:y + h, x:x + w])
            # TEST
            # rect = cv2.rectangle(thresh, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # cv2.imshow('rect', rect)

    # Load model
    model = tf.keras.models.load_model("..\\models\\channo_v0.4.keras")

    # Define result list
    result = []

    # TEST
    filtered_images = []

    for extracted_symbol in extracted_symbols:
        # Define the desired size of the square image (AxA)
        desired_size = max(extracted_symbol.shape) + 15  # offset

        # Create a blank square image of the desired size
        resized_image = np.ones((desired_size, desired_size), dtype=np.uint8) * 255

        # Calculate the position to place the original image in the center
        x_offset = (desired_size - extracted_symbol.shape[1]) // 2
        y_offset = (desired_size - extracted_symbol.shape[0]) // 2

        # Place the original image in the center of the blank square image
        resized_image[y_offset:y_offset + extracted_symbol.shape[0], x_offset:x_offset + extracted_symbol.shape[1]] = extracted_symbol

        # Downscale the resized image to the target size (e.g., 28x28)
        downscaled_image = cv2.resize(resized_image, (28, 28))

        # Invert, normalize and reshape image to give input our model
        filtered_image = 255 - downscaled_image             # Invert

        # TEST
        # filtered_images.append(cv2.GaussianBlur(filtered_image, (5, 5), 0))
        filtered_images.append(filtered_image)

        filtered_image = filtered_image / 255.0             # Normalize
        filtered_image = filtered_image.reshape(1, 28, 28)  # Reshape

        # Pass filtered image to our model
        predictions = model.predict(filtered_image)
        predicted_class = np.argmax(predictions[0])
        result.append(predicted_class)

    # TEST
    # print(len(filtered_images))
    # plot_multiple(filtered_images, 'row')

    # TEST
    # Display the original image with detected lines
    # cv2.imshow('Detected lines', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return result


In [ ]:
def map_to_move_as_string(values):
    custom_mapping = {
        0:  '0',
        1:  '1',
        2:  '2',
        3:  '3',
        4:  '4',
        5:  '5',
        6:  '6',
        7:  '7',
        8:  '8',
        9:  'a',
        10: 'b',
        11: 'c',
        12: 'd',
        13: 'e',
        14: 'f',
        15: 'g',
        16: 'h',
        17: 'K',
        18: 'Q',
        19: 'R',
        20: 'B',
        21: 'N',
        22: '-',
        23: 'x',
        24: '+',
        25: '#'
    }
    result = ''
    for value in values:
        result += custom_mapping.get(value)
    return result

In [ ]:
# Read the image
image = cv2.imread('images\\sheets\\example_sheet.png')

# Converts bgr to rgb color scale, default reads as bgr why?
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Result matrix
player_moves = [[], []]

vertical_sliced_images = slice_vertical(image)
for i, vertical_sliced_image in enumerate(vertical_sliced_images):
    horizontal_sliced_images = slice_horizontal(vertical_sliced_image)
    # plot_multiple(horizontal_sliced_images, 'column')
    for horizontal_sliced_image in horizontal_sliced_images:
        move_as_int_list = predict_move(horizontal_sliced_image)
        player_moves[i % 2].append(map_to_move_as_string(move_as_int_list))
    # move_as_int_list = predict_move(horizontal_sliced_images[6])
    # player_moves[i % 2].append(move_as_int_list)
    # break

print('\nWhite Moves')
print(player_moves[0])

print('\nBlack Moves')
print(player_moves[1])